## RainFall Data Analysis

In [2]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

### Download Partitioned Data

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/rainfall/partitions/"


# Retrieve the article metadata
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want


# Zip to Folder Unzip
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])
        
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

### Combine Data

In [3]:
# Read the data
files_to_combine = glob.glob(output_directory + "*.csv")
files_to_combine.remove(output_directory + "observed_daily_rainfall_SYD.csv")
df = pd.concat(
    (pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[^\/&\\]+(?=_daily_rainfall_NSW\.)', file)[0])
                for file in files_to_combine)
    )

# Save the combined data
data_path = "../data/rainfall/"
os.makedirs(data_path + "combined/", exist_ok=True)
df.to_csv(data_path + "combined/rainfall_data.csv")

### Loading Combined Data and Simple EDA

In [5]:
# Read the data
data = pd.read_csv("../data/rainfall/combined/rainfall_data.csv")

In [39]:
# Testing different loading methods for EDA timing
import sys
# size of 'data' in gigabytes
sys.getsizeof(data)/(1e9)

11.448609766

In [7]:
data.dtypes

time              object
lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

#### No data manipulation

In [50]:
%%timeit -n 3 -r 2 # run statement 3 times for 2 repetitions
data.value_counts()

data.describe()

data.nunique()

3min ± 3.58 s per loop (mean ± std. dev. of 2 runs, 3 loops each)


#### Modifying Data Types

In [52]:
# Changing data types
## Changed 'time' from `object` to `datetime.datetime`
data['time'] = pd.to_datetime(data['time'])
data[['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']] = data[['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']].astype('float32')
sys.getsizeof(data)/(1e9) #size of modified data in GB

5.963029062

In [53]:
%%timeit -n 3 -r 2 # run statement 3 times for 2 repetitions
data.value_counts()

data.describe()

data.nunique()

2min 31s ± 315 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


#### Loading Select Columns

In [9]:
# CLEAR MODIFIED DATAFRAME AND RELOAD
del data

data = pd.read_csv("../data/rainfall/combined/rainfall_data.csv")

In [5]:
# Using only necessary columns (time, rainfall, model), location columns can be loaded in later if needed

data = data[['time', 'rain (mm/day)', 'model']]
sys.getsizeof(data)/(1e9) # Very interesting to see here, the location columns account for only ~10% of the total object size

9.44963879

In [56]:
%%timeit -n 3 -r 2 # run statement 3 times for 2 repetitions
data.value_counts()

data.describe()

data.nunique()

1min 53s ± 274 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [12]:
# convert time to datetime year only
data['time'] = pd.to_datetime(data['time'])
data['year'] = data['time'].dt.year
# group data by year and model
data_grouped = data.groupby(['year', 'model']).mean()
data_grouped


lat_min    lat_max   lon_min   lon_max  rain (mm/day)
year model                                                                  
1889 ACCESS-CM2     -33.125000 -31.875000  146.2500  148.1250       1.398767
     ACCESS-ESM1-5  -33.125000 -31.875000  147.1875  149.0625       1.856297
     AWI-ESM-1-1-LR -33.574618 -31.709369  147.1875  149.0625       2.280675
     BCC-CSM2-MR    -33.083931 -31.962443  146.8125  147.9375       2.075723
     BCC-ESM1       -33.487230 -30.696650  146.2500  149.0625       1.820705
...                        ...        ...       ...       ...            ...
2014 NESM3          -33.750000 -31.875000  148.1250  150.0000       2.118067
     NorESM2-LM     -33.157895 -31.263158  146.2500  148.7500       2.304739
     NorESM2-MM     -32.984293 -32.041885  146.8750  148.1250       1.859380
     SAM0-UNICON    -32.984293 -32.041885  146.8750  148.1250       2.228741
     TaiESM1        -32.984293 -32.041885  146.8750  148.1250       2.124587

[3402 rows x 5 columns]

In [14]:
# get the mean rainfall for each year and model
data_grouped.reset_index(inplace=True)
data_grouped.head()

,year,model,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889,ACCESS-CM2,-33.125000,-31.875000,146.2500,148.1250,1.398767
1,1889,ACCESS-ESM1-5,-33.125000,-31.875000,147.1875,149.0625,1.856297
2,1889,AWI-ESM-1-1-LR,-33.574618,-31.709369,147.1875,149.0625,2.280675
3,1889,BCC-CSM2-MR,-33.083931,-31.962443,146.8125,147.9375,2.075723
4,1889,BCC-ESM1,-33.487230,-30.696650,146.2500,149.0625,1.820705


In [19]:
# plot the data and facet by model
# set the plot size
import altair as alt
alt.data_transformers.disable_max_rows()
alt.Chart(data_grouped).mark_line().encode(
    x='year:O',
    y='rain (mm/day):Q',
    color='model:N'
).properties(
    width=600,
    height=400
).facet(
    row='model:N'
)


alt.FacetChart(...)

#### Conclusion and Discussion

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken for Each Type of Memory Reduction |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Sam   |    macOS Catalina      |16GB | 2.8 GHz 4 Core i7   |  Yes   | 3mins, 2.5mins, 1.9mins     |
| Member 2    |                  |     |           |        |            |
| Member 3    |                  |     |           |        |            |
| Member 4    |                  |     |           |        |            |

I attempted to show some plots but unless I loaded only parts of the dataset, they would not execute. In terms of memory space, changing the data types was the best at reducing the size of the object. This is due to the changing of the numerical data from float64 to float32, which is effectively half the space. The trade-off for doing this is that now our data is less precise than before.

In [20]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra


In [21]:
%load_ext rpy2.ipython


In [22]:
%%R
suppressMessages(library(dplyr, warn.conflicts=FALSE))
suppressMessages(library(arrow, warn.conflicts=FALSE))


In [23]:
%%time
df = pd.read_csv("../data/rainfall/combined/rainfall_data.csv")


CPU times: user 30.5 s, sys: 7.25 s, total: 37.7 s
Wall time: 41 s


In [25]:
%%time
data_path = "../data/rainfall/"
df.to_feather(data_path + "combined/rainfall_data.feather")


CPU times: user 4.81 s, sys: 3.82 s, total: 8.63 s
Wall time: 7.7 s


In [26]:
%%time
%%R
ds <- open_dataset("../data/rainfall/combined/rainfall_data.feather",
                 format="feather")


CPU times: user 37 ms, sys: 62.6 ms, total: 99.5 ms
Wall time: 183 ms
